In [1]:
import numpy as np
from scipy import constants as c
from scipy import special as sp
from iminuit import Minuit, cost
from argparse import Namespace
import pandas as pd
import os
import matplotlib.pyplot as plt
import h5py as h5
from scipy.optimize import curve_fit

In [2]:
KB = c.Boltzmann
hbar = c.Planck


In [3]:
def QT(T,d0, q0, alpha, omega):
    csi = (hbar * omega)/(2*KB*T)
    num = np.exp(-d0/(KB*T))*np.sinh(csi)*sp.kn(0,csi)
    den = np.pi*(1-2*np.exp(-d0/(KB*T))*np.exp(-csi)*sp.iv(0,-csi))
    return num/den * (alpha/2) + 1/q0

def kondo(T,d0, q0, alpha, omega, b, T_k):
    csi = (hbar * omega)/(2*KB*T)
    num = np.exp(-d0/(KB*T))*np.sinh(csi)*sp.kn(0,csi)
    den = np.pi*(1-2*np.exp(-d0/(KB*T))*np.exp(-csi)*sp.iv(0,-csi))
    return num/den * (alpha/2) + 1/q0 - b*np.log(T/T_k)

def retta(x, m, q):
    return m*x + q

def resonance(x, x0, Q, Q_c, phi0):
    num = np.exp(1j*phi0)
    den = 1 + 2*1j*Q*(x-x0)/x0
    return abs(1 - (Q/Q_c)*(num/den))

def model(x,m,q,x0,Q,Q_c,phi0):
    return retta(x,m,q) * resonance(x,x0,Q,Q_c,phi0)

def FWHM(x,y):
    x = np.array(x)
    y = np.array(y)
    half = max(y) - (max(y)-min(y)) / 2.
    minimums = x[np.where(y < half)]
    if len(minimums)==0: 
        print('Minimum not found!')
        return 0,1
    else:
        return (max(minimums) - min(minimums)),0

def mod(x,y):
    return np.sqrt(x**2 + y**2)
    
def Q_stima(x0,w):
    if (((x0 is None) or (w is None)) or w==0):
        print('Q_stima: None detected!')
        return 0
    else:
        return x0/(2*w)
    
def Sort_Tuple(tup):
 
    # getting length of list of tuples
    lst = len(tup)
    for i in range(0, lst):
        for j in range(0, lst-i-1):
            if (tup[j][1] > tup[j + 1][1]):
                temp = tup[j]
                tup[j] = tup[j + 1]
                tup[j + 1] = temp
    return tup


def compute_chi2(func,x, y, sigma, p):
    sum = 0
    for (i,j,s) in zip(x,y,sigma):
        sum += ((j-func(i,*p))/s)**2
    return sum





def plotter(dfs, t, func, p, c, rows, columns, fill):

    fig, ax = plt.subplots(rows, columns, figsize=(30,20))

    for row in range(rows):
        for col in range(columns):
            i = row*columns + col


            chi2 = compute_chi2(func, np.array(dfs[i]['f']),np.array(dfs[i]['S21']), np.array(dfs[i]['S21'])*1e-3, p[i])
            ndof = len(dfs)
            title = [
                f"chi2/ndof = {chi2:.1f}/{ndof}={chi2/ndof:.1f}",
            ]

            for (l,value) in enumerate(p[i]):
                title.append(
                    f"{value:.4f} +/- {np.diag(c[i])[l]:.4f}"
                )


            ax[row][col].scatter(np.array(dfs[i]['f'])*1e-9, np.array(dfs[i]['S21']), marker='.', s=1, label='Raw Data', color='black')
            ax[row][col].plot(np.array(dfs[i]['f'])*1e-9, func(np.array(dfs[i]['f']), *p[i]), label='Model', color='red')
            if (fill==True):
                ax[row][col].fill_between(np.array(dfs[i]['f'])*1e-9, np.array(dfs[i]['S21'])*(1-(1e-3)), np.array(dfs[i]['S21'])*(1+1e-3), alpha=0.5, color='blue')
            ax[row][col].set_facecolor('bisque')
            ax[row][col].set_xlabel(r'$\nu$ [GHz]')
            ax[row][col].set_ylabel(r'$|S_{21}|$')
            ax[row][col].legend(title='\n'.join(title))
            ax[row][col].set_title('T: '+str(t[i]))
            ax[row][col].grid()
            fig.tight_layout()

    return fig

def read_files(dir = r'C:\Users\kid\SynologyDrive\Lab2023\KIDs\QTLab2324\IRdetection\Instruments\Test_data\data_19_02_set1\\'):

    dfs, dictionaries = [], []

    for folder in os.listdir(dir):
        if ('T' in folder):
            parent = dir+folder+'\\'
            dictionaries = []
            for (i,filename) in enumerate(os.listdir(parent)):
                my_dest = filename.replace('.h5','.hdf5')
                my_source = parent + filename
                my_dest = parent + my_dest
                os.rename(my_source, my_dest)
                with h5.File(my_dest, 'r') as r:
                    group = r.require_group(name='raw_data')
                    dic = dict()
                    for c in group.items():
                        dic[c[0]] = c[1][:]
                dictionaries.append(dic)
        dfs.append(dictionaries)
    return dfs

In [4]:
dfs = read_files() 

In [6]:
f0 = []
for i in range(12):
    for j in range(4):
        dfs[i][j]['S21'] = 20*np.log10(np.sqrt(np.array(dfs[i][j]['i'])**2 + np.array(dfs[i][j]['q'])**2))
        f0.append(float(np.array(dfs[i][j]['f'])[np.where(np.array(dfs[i][j]['S21']) == np.min(np.array(dfs[i][j]['S21'])))[0]]))

In [7]:
f1 = [14.4827e9, 14.8814e9, 15.1563e9, 15.7100e9]

In [12]:
def alpha(f0, f1):
    return 1-(f0/f1)**2

In [14]:
for i in range(4):
    print(alpha(f0[12*i], f1[0]))

0.8637057238431085
0.86368024743377
0.8637063607228376
0.863733745143523
